In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import cv2
import random


In [2]:
# Absolute main path
main_path = '../data/'

# Read JSON file into a DataFrame with unprocessed instance col
wlas_df = pd.read_json(main_path + 'WLASL_v0.3.json')

print(wlas_df.head())
print(wlas_df.shape)


      gloss                                          instances
0      book  [{'bbox': [385, 37, 885, 720], 'fps': 25, 'fra...
1     drink  [{'bbox': [551, 68, 1350, 1080], 'fps': 25, 'f...
2  computer  [{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...
3    before  [{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...
4     chair  [{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...
(2000, 2)


In [3]:
def get_videos_ids(json_list):
    """
    function to check if the video id is available in the dataset
    and return the viedos ids of the current instance

    Input: instance json list
    Output: list of videos_ids
    """
    videos_list = []
    for ins in json_list:
        video_id = ins['video_id']
        if os.path.exists(f'{main_path}videos/{video_id}.mp4'):
            videos_list.append(video_id)
    return videos_list


In [4]:
def get_json_features(json_list):
    """
    function to check if the video id is available in the dataset
    and return the viedos ids and url or any other featrue of the current instance

    input: instance json list
    output: list of videos_ids
    """
    videos_ids = []
    videos_urls = []
    for ins in json_list:
        video_id = ins['video_id']
        video_url = ins['url']
        if os.path.exists(f'{main_path}videos/{video_id}.mp4'):
            videos_ids.append(video_id)
            videos_urls.append(video_url)
    return videos_ids, videos_urls


In [5]:
# Open JSON file (read only)
with open(main_path+'WLASL_v0.3.json', 'r') as data_file:
    json_data = data_file.read()

instance_json = json.loads(json_data)


In [6]:
# Get available video ids for all rows in wlas_df and add to new col 'videos_id'
wlas_df['videos_ids'] = wlas_df['instances'].apply(get_videos_ids)
wlas_df.head()


,gloss,instances,videos_ids
0,book,"[{'bbox': [385, 37, 885, 720], 'fps': 25, 'fra...","[69241, 07069, 07068, 07070, 07099, 07074]"
1,drink,"[{'bbox': [551, 68, 1350, 1080], 'fps': 25, 'f...","[69302, 65539, 17710, 17733, 65540, 17734, 177..."
2,computer,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...","[12328, 12312, 12311, 12338, 12313, 12314, 123..."
3,before,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...","[05728, 05749, 05750, 05729, 05730, 65167, 057..."
4,chair,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...","[09848, 09869, 09849, 09850, 09851, 65328, 09854]"


In [7]:
# Create separate DataFrame for available information in each instance
features_df = pd.DataFrame(columns=['word', 'video_id', 'url'])

for row in wlas_df.iterrows():
    # Extract ids and urls for each row
    ids, urls = get_json_features(row[1][1])
    # Initialize a list matching the length (n) of found ids containing the word
    word = [row[1][0]] * len(ids)
    # Using zip to create new df with:
    # n * word in gloss col (e.g. 6 * book)
    # Unique id and url in ids and url col respectively
    df = pd.DataFrame(list(zip(word, ids, urls)), columns = features_df.columns)
    # Append temporary df to feature_df
    features_df = pd.concat([features_df, df], ignore_index=True)

# Renaming index col to index
features_df.index.name = 'index'
features_df.head()


,word,video_id,url
index,,,
0,book,69241,http://aslbricks.org/New/ASL-Videos/book.mp4
1,book,07069,https://signstock.blob.core.windows.net/signsc...
2,book,07068,https://s3-us-west-1.amazonaws.com/files.start...
3,book,07070,https://media.asldeafined.com/vocabulary/14666...
4,book,07099,http://www.aslsearch.com/signs/videos/book.mp4


In [8]:
selected_words = [
    'like',
    'work',
    'play',
    'take',
    'call',
    'go',
    'study',
    'give',
    'write',
    'yesterday',
    'far',
    'hot',
    'cold',
    'good',
    'bad',
    'computer',
    'apple',
    'doctor',
    'family',
    'dog'
]


In [9]:
selected_df = features_df[features_df['word'].isin(selected_words)]
selected_df


,word,video_id,url
index,,,
21,computer,12328,https://www.signingsavvy.com/signs/mp4/6/6326.mp4
22,computer,12312,https://s3-us-west-1.amazonaws.com/files.start...
23,computer,12311,https://media.spreadthesign.com/video/mp4/13/5...
24,computer,12338,http://www.aslsearch.com/signs/videos/computer...
25,computer,12313,https://s3-us-west-1.amazonaws.com/files.start...
...,...,...,...
1510,good,25076,http://www.aslsearch.com/signs/videos/good.mp4
1511,good,25067,https://s3-us-west-1.amazonaws.com/files.start...
1512,good,25068,https://signstock.blob.core.windows.net/signsc...


In [10]:
for video_id in selected_df['video_id']:
    if os.path.exists(f'{main_path}videos/{video_id}.mp4'):
        cap = cv2.VideoCapture(f'{main_path}videos/{video_id}.mp4')
        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        selected_df.loc[selected_df['video_id'] == video_id, ['video_length']] = int(length)
    pass

selected_df = selected_df.reset_index(drop=True)
selected_df


,word,video_id,url,video_length
0,computer,12328,https://www.signingsavvy.com/signs/mp4/6/6326.mp4,88.0
1,computer,12312,https://s3-us-west-1.amazonaws.com/files.start...,101.0
2,computer,12311,https://media.spreadthesign.com/video/mp4/13/5...,72.0
3,computer,12338,http://www.aslsearch.com/signs/videos/computer...,107.0
4,computer,12313,https://s3-us-west-1.amazonaws.com/files.start...,81.0
...,...,...,...,...
214,good,25076,http://www.aslsearch.com/signs/videos/good.mp4,86.0
215,good,25067,https://s3-us-west-1.amazonaws.com/files.start...,74.0
216,good,25068,https://signstock.blob.core.windows.net/signsc...,60.0
217,good,25069,https://signstock.blob.core.windows.net/signsc...,21.0


In [11]:
selected_df.describe()


,video_length
count,219.000000
mean,65.228311
std,23.406910
min,20.000000
25%,46.000000
50%,64.000000
75%,81.000000
max,148.000000


In [13]:
# Frame sampling parameters
frames_per_video = 10
target_size = (150, 150)
# Initialize empty array of desired shape
results = np.empty((219, frames_per_video, *target_size, 3), dtype=np.uint8)

# Function to perform frame sampling
def sample_frames(video_path, frames_per_video, total_frames):
    frames = []
    cap = cv2.VideoCapture(video_path)

    frame_indices = []

    while len(set(frame_indices)) != frames_per_video:
        frame_indices = sorted(np.random.uniform(0, total_frames-5, frames_per_video).astype(int))

    frame_counter = 0

    print(frame_indices, total_frames)

    try:
        while cap.isOpened():
            ret, frame = cap.read()

            if not ret:
                break

            if frame_counter in frame_indices:
                # Resize frame to required size
                frame = cv2.resize(frame, target_size)
                # CV2 output BGR -> converting to RGB
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                # Append to list of frames
                frames.append(frame_rgb)

            frame_counter += 1

            if len(frames) == frames_per_video:
                break

    finally:
        cap.release()

    return frames


In [22]:
np.random.seed(10)

for i, row in selected_df.iterrows():
    video_id = row['video_id']
    total_frames = row['video_length']
    video_path = f'../data/videos/{video_id}.mp4'

    sampled_frames = sample_frames(video_path, frames_per_video, total_frames)
    print(np.array(sampled_frames).shape)
    print(i)
    # Assign sampled frames to results array
    results[i] = np.array(sampled_frames)


[1, 7, 14, 16, 18, 41, 52, 62, 63, 64] 88.0
(10, 150, 150, 3)
0
[0, 28, 49, 58, 65, 68, 69, 78, 88, 91] 101.0
(10, 150, 150, 3)
1
[3, 6, 7, 20, 21, 34, 41, 53, 55, 60] 72.0
(10, 150, 150, 3)
2
[16, 20, 30, 33, 35, 55, 76, 83, 87, 90] 107.0
(10, 150, 150, 3)
3
[2, 6, 23, 25, 27, 40, 44, 45, 58, 68] 81.0
(10, 150, 150, 3)
4
[0, 1, 2, 7, 8, 11, 12, 32, 34, 36] 43.0
(10, 150, 150, 3)
5
[1, 6, 13, 14, 20, 23, 36, 42, 44, 47] 57.0
(10, 150, 150, 3)
6
[4, 13, 14, 16, 19, 20, 21, 30, 32, 33] 43.0
(10, 150, 150, 3)
7
[2, 5, 9, 12, 22, 32, 39, 41, 45, 46] 53.0
(10, 150, 150, 3)
8
[0, 4, 5, 12, 14, 23, 25, 30, 37, 38] 45.0
(10, 150, 150, 3)
9
[0, 1, 2, 7, 12, 19, 23, 27, 33, 34] 40.0
(10, 150, 150, 3)
10
[5, 13, 14, 20, 24, 43, 49, 67, 77, 96] 102.0
(10, 150, 150, 3)
11
[6, 12, 19, 37, 39, 40, 41, 49, 57, 60] 73.0
(10, 150, 150, 3)
12
[1, 11, 18, 19, 22, 23, 27, 28, 29, 31] 46.0
(10, 150, 150, 3)
13
[2, 4, 6, 9, 19, 29, 34, 38, 45, 51] 67.0
(10, 150, 150, 3)
14
[13, 24, 29, 31, 36, 37, 39, 43, 44

[h264 @ 0x559d66692640] Invalid NAL unit size (745 > 472).
[h264 @ 0x559d66692640] Error splitting the input into NAL units.
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x559d65a2cbc0] stream 1, offset 0x3b468: partial file


(10, 150, 150, 3)
165
[0, 1, 2, 4, 5, 6, 7, 9, 12, 14] 20.0
(10, 150, 150, 3)
166
[1, 2, 10, 21, 29, 37, 43, 44, 49, 72] 79.0
(10, 150, 150, 3)
167
[0, 1, 7, 32, 39, 49, 57, 73, 78, 87] 94.0
(10, 150, 150, 3)
168
[0, 3, 6, 7, 8, 18, 19, 22, 23, 26] 34.0
(10, 150, 150, 3)
169
[1, 10, 11, 15, 31, 35, 39, 41, 42, 49] 55.0
(10, 150, 150, 3)
170
[4, 8, 9, 11, 14, 37, 39, 40, 46, 59] 71.0
(10, 150, 150, 3)
171
[1, 4, 6, 7, 8, 15, 16, 17, 24, 28] 40.0
(10, 150, 150, 3)
172
[4, 7, 17, 36, 37, 39, 43, 57, 70, 77] 86.0
(10, 150, 150, 3)
173
[1, 2, 9, 15, 17, 23, 30, 36, 39, 43] 56.0
(10, 150, 150, 3)
174
[0, 1, 3, 4, 6, 7, 12, 19, 23, 25] 35.0
(10, 150, 150, 3)
175
[0, 7, 8, 11, 12, 19, 21, 22, 23, 24] 35.0
(10, 150, 150, 3)
176
[9, 10, 12, 17, 18, 20, 29, 30, 33, 43] 50.0
(10, 150, 150, 3)
177
[5, 36, 45, 50, 53, 57, 73, 74, 80, 81] 88.0
(10, 150, 150, 3)
178
[0, 2, 4, 10, 11, 13, 14, 19, 24, 29] 35.0
(10, 150, 150, 3)
179
[1, 4, 7, 9, 11, 12, 16, 17, 19, 21] 31.0
(10, 150, 150, 3)
180
[2, 4, 9

In [23]:
results.shape


(219, 10, 150, 150, 3)

In [24]:
results


array([[[[[255, 255, 255],
          [255, 255, 255],
          [255, 255, 255],
          ...,
          [255, 255, 255],
          [255, 255, 255],
          [255, 255, 255]],

         [[255, 255, 255],
          [255, 255, 255],
          [255, 255, 255],
          ...,
          [255, 255, 255],
          [255, 255, 255],
          [255, 255, 255]],

         [[255, 255, 255],
          [255, 255, 255],
          [255, 255, 255],
          ...,
          [255, 255, 255],
          [255, 255, 255],
          [255, 255, 255]],

         ...,

         [[251, 255, 255],
          [255, 252, 255],
          [254, 254, 254],
          ...,
          [228, 231, 236],
          [225, 233, 237],
          [250, 250, 253]],

         [[253, 254, 255],
          [255, 253, 255],
          [255, 254, 255],
          ...,
          [254, 255, 255],
          [251, 255, 255],
          [255, 255, 255]],

         [[255, 254, 255],
          [255, 253, 255],
          [255, 254, 255],
         